In [74]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Encode string labels to integer values
nltk.data.path.append("/path/to/nltk_data")



In [75]:
data = pd.read_csv('Student Mental Wellness Survey (Responses) - Form responses 1.csv')

data

,Timestamp,Age,Which state are you from?,Gender,Grade Level/Year in School,What are you currently studying?,How often have you been feeling sad or hopeless in the past two weeks?,"How often have you been feeling nervous, anxious, or on edge in the past two weeks?",Have you experienced changes in your appetite or weight in the past two weeks?,"Have you had trouble sleeping (either difficulty falling asleep, staying asleep, or waking up too early) in the past two weeks?",Have you been feeling unusually fatigued or lacking in energy in the past two weeks?,Have you had trouble concentrating on things such as schoolwork or other activities in the past two weeks?,Have you had recurring thoughts of self harm in the past two weeks?,"Have you experienced any physical symptoms such as racing heart, trembling, sweating, or shortness of breath without any apparent cause in the past two weeks?",Have you noticed any changes in your academic performance or attendance due to emotional distress in the past two weeks?,Is there anything else you would like to share about your emotional well-being or any concerns you have?,Do you think you have depression or anxiety?,Have you ever been diagnosed with depression or anxiety by a healthcare professional?
0,05/03/2024 22:23:50,20,Odisha,Male,Bachelor's 3rd Year,Computer Science,Everyday,Often,Weight gain,Often,Everyday,Everyday,Everyday,Rarely,No change,Nothing,Yes,No
1,05/03/2024 22:31:08,69,Liquid,Male,Bachelor's 3rd Year,Computer Science,Everyday,Everyday,No change,Everyday,Everyday,Sometimes,Everyday,Everyday,No change,nope,Yes,Yes
2,13/03/2024 12:53:08,20,Hariyana,Male,Bachelor's 3rd Year,BBA,Sometimes,Rarely,No change,No,No,No,No,No,No change,No,No,Yes
3,13/03/2024 12:54:03,21,UP,Male,Bachelor's 4th Year,CSE,No,No,No change,No,No,No,No,No,No change,NaN,Yes,Yes
4,13/03/2024 12:54:48,56,Delhi,Female,Bachelor's 2nd Year,BA,Rarely,Rarely,Increased appetite,Rarely,Rarely,Rarely,Rarely,Rarely,Slight decline,NaN,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,28/03/2024 11:39:31,20,Punjab,Female,Bachelor's 3rd Year,Bsc IT,Sometimes,Sometimes,No change,Sometimes,Rarely,Sometimes,No,Sometimes,No change,No,I am not sure.,No
178,28/03/2024 11:42:48,20,Maharashtra,Female,Bachelor's 3rd Year,Bcom accounting and finance,Often,Sometimes,No change,Rarely,No,Rarely,No,No,Significant decline,NaN,I am not sure.,No
179,28/03/2024 16:24:49,19,Maharashtra,Female,Bachelor's 2nd Year,Bsc nursing,No,No,Increased appetite,No,Sometimes,Rarely,No,No,No change,None i am good,No,No
180,28/03/2024 18:26:02,18,maharashtra,Female,Bachelor's 1st Year,fashion,Sometimes,Often,Decreased appetite,Often,Rarely,Sometimes,No,Sometimes,No change,NaN,Yes,No


In [76]:
data.columns = [f'X{i}' for i in range(1, len(data.columns) + 1)]
data = data.iloc[:, 1:]
data

,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
0,20,Odisha,Male,Bachelor's 3rd Year,Computer Science,Everyday,Often,Weight gain,Often,Everyday,Everyday,Everyday,Rarely,No change,Nothing,Yes,No
1,69,Liquid,Male,Bachelor's 3rd Year,Computer Science,Everyday,Everyday,No change,Everyday,Everyday,Sometimes,Everyday,Everyday,No change,nope,Yes,Yes
2,20,Hariyana,Male,Bachelor's 3rd Year,BBA,Sometimes,Rarely,No change,No,No,No,No,No,No change,No,No,Yes
3,21,UP,Male,Bachelor's 4th Year,CSE,No,No,No change,No,No,No,No,No,No change,NaN,Yes,Yes
4,56,Delhi,Female,Bachelor's 2nd Year,BA,Rarely,Rarely,Increased appetite,Rarely,Rarely,Rarely,Rarely,Rarely,Slight decline,NaN,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,20,Punjab,Female,Bachelor's 3rd Year,Bsc IT,Sometimes,Sometimes,No change,Sometimes,Rarely,Sometimes,No,Sometimes,No change,No,I am not sure.,No
178,20,Maharashtra,Female,Bachelor's 3rd Year,Bcom accounting and finance,Often,Sometimes,No change,Rarely,No,Rarely,No,No,Significant decline,NaN,I am not sure.,No
179,19,Maharashtra,Female,Bachelor's 2nd Year,Bsc nursing,No,No,Increased appetite,No,Sometimes,Rarely,No,No,No change,None i am good,No,No
180,18,maharashtra,Female,Bachelor's 1st Year,fashion,Sometimes,Often,Decreased appetite,Often,Rarely,Sometimes,No,Sometimes,No change,NaN,Yes,No


In [77]:
data['X3'].unique()

array(['Odisha', 'Liquid', 'Hariyana ', 'UP ', 'Delhi', 'Rajisthan',
       'haryana', 'Bihar', 'Up', 'Uttar pradesh ', 'Uttar Pradesh ',
       'Maharashtra ', 'Bihar ', 'Uttar pradesh', 'West Bengal ',
       'Maharastra ', '-', 'Madhya Pradesh ', 'Jharkhand ', 'Tamil Nadu ',
       'UP', 'Madhya pradesh ', 'Uttar Pradesh', 'Haryana', 'Odisha ',
       'Jharkhand', 'Rajasthan', 'Chhattisgarh', 'HARYANA ',
       'Chhattisgarh ', 'MP', 'Karnataka ', 'Maharashtra',
       'Not from India', 'West Bengal', 'Gujrat ', 'Delhi NCR',
       'Madhya Pradesh', 'UttarPradesh', 'uttar Pradesh', 'Assam',
       'maharashtra ', 'Karnataka', 'Goa', 'delhi ', 'mp', 'Mahrashtra',
       'Punjab'], dtype=object)

In [78]:
data.loc[:, 'X3'] = data['X3'].str.strip()
data['X3'].unique()

array(['Odisha', 'Liquid', 'Hariyana', 'UP', 'Delhi', 'Rajisthan',
       'haryana', 'Bihar', 'Up', 'Uttar pradesh', 'Uttar Pradesh',
       'Maharashtra', 'West Bengal', 'Maharastra', '-', 'Madhya Pradesh',
       'Jharkhand', 'Tamil Nadu', 'Madhya pradesh', 'Haryana',
       'Rajasthan', 'Chhattisgarh', 'HARYANA', 'MP', 'Karnataka',
       'Not from India', 'Gujrat', 'Delhi NCR', 'UttarPradesh',
       'uttar Pradesh', 'Assam', 'maharashtra', 'Goa', 'delhi', 'mp',
       'Mahrashtra', 'Punjab'], dtype=object)

In [79]:
data.loc[:, 'X3'] = data['X3'].str.strip().str.lower()
replacements = {'up': 'Uttar Pradesh', 'uttarpradesh': 'Uttar Pradesh', 'mp': 'Madhya Pradesh', 'delhi ncr': 'Delhi','mahrashtra':'Maharashtra','rajisthan' : 'Rajasthan','hariyana' : 'Haryana','maharastra':'Maharashtra'}
data.loc[:, 'X3'] = data['X3'].replace(replacements)

data.loc[:, 'X3'] = data['X3'].apply(lambda x: x.title())

data['X3'].value_counts()

X3
Odisha            36
Maharashtra       35
Madhya Pradesh    19
Jharkhand         18
Bihar             18
Uttar Pradesh     14
Haryana            7
Assam              7
West Bengal        6
Delhi              6
Chhattisgarh       4
Karnataka          3
Rajasthan          2
-                  1
Liquid             1
Tamil Nadu         1
Not From India     1
Gujrat             1
Goa                1
Punjab             1
Name: count, dtype: int64

In [80]:
data['X16'] = data['X16'].astype(str)

sia = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


negative_responses = ['no', 'nope', 'nothing', 'na','nope.','nah','not at all','no.', ]


data['X16_sentiment'] = data['X16'].apply(lambda x: 0.0 if any(token in [stemmer.stem(lemmatizer.lemmatize(word)) for word in nltk.word_tokenize(x.lower())] for token in negative_responses) else sia.polarity_scores(x)['compound'] if isinstance(x, str) else 0.0)

columns_to_encode = [col for col in data.columns if col not in ['X2', 'X6', 'X17', 'X16', 'X16_sentiment']]
data_encoded = pd.get_dummies(data, columns=columns_to_encode, drop_first=False)

data = data_encoded.drop(columns=['X16','X6'])
data


,X2,X17,X16_sentiment,X3_-,X3_Assam,X3_Bihar,X3_Chhattisgarh,X3_Delhi,X3_Goa,X3_Gujrat,...,X14_Everyday,X14_No,X14_Often,X14_Rarely,X14_Sometimes,X15_No change,X15_Significant decline,X15_Slight decline,X18_No,X18_Yes
0,20,Yes,0.0000,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,True,False
1,69,Yes,0.0000,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,True
2,20,No,0.0000,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,True
3,21,Yes,0.0000,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,True
4,56,No,0.0000,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,20,I am not sure.,0.0000,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,True,False
178,20,I am not sure.,0.0000,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,True,False
179,19,No,-0.3412,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,True,False
180,18,Yes,0.0000,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,True,False


In [81]:
data['X16_sentiment'].value_counts()

X16_sentiment
 0.0000    160
-0.4215      3
-0.7430      2
 0.3400      1
-0.6369      1
-0.4201      1
-0.1027      1
-0.2500      1
 0.3818      1
-0.1832      1
 0.3164      1
-0.2732      1
 0.8715      1
 0.4310      1
 0.3382      1
 0.5837      1
-0.8889      1
 0.6249      1
-0.5106      1
-0.3412      1
Name: count, dtype: int64

In [82]:
X = data.drop(columns=['X17'])  
y = data['X17']  

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X, y)

with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(rf_classifier, file)

In [83]:
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X, y)

# Saving Decision Tree Classifier to a file
with open('decision_tree_model.pkl', 'wb') as file:
    pickle.dump(dt_classifier, file)

In [84]:

# Assuming X and y are your feature matrix and target vector respectively
# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Standardize your features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define your neural network architecture
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # Output layer with softmax activation for multiclass classification
])

# Compile the model
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the neural network
nn_model.fit(X_train_scaled, y_train_encoded, epochs=300, batch_size=50, validation_split=0.2)  # Adjust epochs, batch_size, and validation_split as needed

# Evaluate the model
test_loss, test_accuracy = nn_model.evaluate(X_test_scaled, y_test_encoded)
print("Test Accuracy:", test_accuracy)



c:\Ananconda\envs\SELF\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.1866 - loss: 1.3423 - val_accuracy: 0.2414 - val_loss: 1.3801
Epoch 2/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2981 - loss: 1.2083 - val_accuracy: 0.2414 - val_loss: 1.2930
Epoch 3/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3926 - loss: 1.0967 - val_accuracy: 0.2759 - val_loss: 1.2254
Epoch 4/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5168 - loss: 0.9850 - val_accuracy: 0.4138 - val_loss: 1.1708
Epoch 5/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6097 - loss: 0.9187 - val_accuracy: 0.4483 - val_loss: 1.1267
Epoch 6/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6619 - loss: 0.8580 - val_accuracy: 0.5172 - val_loss: 1.0893
Epoch 7/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7471 - loss: 0.7944 - val_accuracy: 0.5517 - val_loss: 1.0555
Epoch 8/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7711 - loss: 0.7523 - val_accuracy: 0.5517 - val_loss